Tool updated: 2024-08-08, Henrik Loecke

Tool summary:
This tool creates RAWN sheets, based on the MIKE+ model and MPF population file. It traces through the model to list the upstream catchments of each manhole (in flow splits, each split direction will both get 100% of the upstream flow). It then uses the MPF sheets to summarise population and residential/ICI areas for each catchment. Aggregated catchment contributions to each manhole are then used as input to RAWN calculations. For each manhole, the upstream catchments are dissolved together and added to a layer 'Node_Catchment'. This is used as input to 'Map Series' (similar to 'Data Driven Pages' in ArcGIS Desktop). This is used to create one jpg image per manhole, highlighting the manhole and all its upstream catchments. The RAWN calculations and images are added to RAWN spreadsheets which are similar in layout to the original RAWN sheets which are derived from Neural Network. One important difference in RAWN calculations to previous is that no peaking factor is allowed to go below 1.5, as per official RAWN guidelines. Peaking factors that reach the minimum are highlighted in yellow. Also a Google Maps link is added to each sheet. The user can choose between adding RAWN calculations as values or Excel formulas. Excel formulas are recommended for full transparency of the calculations to those using the sheets. However, python calculations are maintained in the code to enable future additional outputs (Power BI, HTML etc.).


In [170]:
#Permanent cell 1
import arcpy
import pandas as pd
import sqlite3
import math
import numpy as np
import os
import sys
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side, numbers
from openpyxl.formatting.rule import FormulaRule
import ctypes
import traceback
import shutil
import subprocess
import gc
MessageBox = ctypes.windll.user32.MessageBoxA

In [171]:
#Permanent cell 2

#Create functions for to use SQL with the model sqlite database.

def sql_to_df(sql,model):
    con = sqlite3.connect(model)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
            cur.execute(sql)
    else:         
        cur.execute(sqls)
    cur.close()
    con.commit()
    con.close()

In [172]:
#Permanent cell 3

#User Input
try:

    model = 'VSA'

    max_steps = 1005 #
    use_formula = True
    transfer_mixed_area_to_commercial = True

    gdb_name = 'RAWN.gdb'
    gdb_name_dissolve = 'RAWN_Dissolve.gdb' #To keep clutter out of main database

    #Options to skip time consuming steps during debug (by setting to False), must be True during production runs.
    run_dissolve = True
    run_jpg = True

    check_scenario = True #Scenario check is for max active altid. In some hypothetical setups it can be incorrect. 
    #                      If you verify the scenario is correct (open model to check) but get the error, set this check to False
    #                      A more robust check may be developed in the future.


    induced_values = [] #Only to be appended to under specific models where required

    if model == 'NSSA':   
        model_output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\04_ANALYSIS_WORK\RAWN_From_Model"
        model_path = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\NSSA_Base_2018pop.sqlite"
        pop_book = r"\\prdsynfile01\LWS_Modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\MPF4_Temp_Hold\NSSA_Master_Population_File_4_No_2237_ResArea.xlsx"
        pop_sheet = 'MPF Update 4'
        scenario = 'Base'
        wwtp_muid = '22602'
        line_exclusions = []
        years = [2060,2070,2080,2090,2100]
        model_version = 89
        excluded_acronyms_csv = ''

    if model == 'FSA':   
        model_output_folder = r'\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\RAWN_From_Model'
        model_path = r"J:\SEWER_AREA_MODELS\FSA\01_MASTER_MODEL\MODEL\FSA_Base_2021pop.sqlite"
        pop_book = r"J:\SEWER_AREA_MODELS\FSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\FSA_Master_Population_File.xlsx"
        pop_sheet = 'MPF Update 17'
        scenario = '2030_Network'
        wwtp_muid = '1162'
        line_exclusions = ['GoldenEar_Dummy_Pump','GoldenEarSSOLink','GoldenEar_SSO_Tank_Cell_Dummy_Link3','MH35_Orifice','44473']
        years = [2060,2070,2080,2090,2100]
        model_version = 154
        excluded_acronyms_csv = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\RAWN_From_Model\FSA_Excluded_Acronyms.csv"
        induced_values.append(['84674','Area_Ind',257.3]) #Vancouver landfill, equivalent area to get 520 L/s PWWF.
        
    if model == 'VSA':   
        model_output_folder = r"J:\SEWER_AREA_MODELS\VSA\04_ANALYSIS_WORK\79. RAWN_From_Model"
        model_path = r"J:\SEWER_AREA_MODELS\VSA\03_SIMULATION_WORK\Key_Flow_HGL_GIS_Sim_VSA\MIKE+_Import\VSA_V304.sqlite"
        pop_book = r"J:\SEWER_AREA_MODELS\VSA\02_MODEL_COMPONENTS\03_NETWORKS\11. NETWORK_UPDATE\O. V304 Updates\VSA_Master_Population_File.xlsx"
        pop_sheet = 'Export'
        scenario = 'Base'
        wwtp_muid = '7478'
        line_exclusions = ['51833','51834','51835','FS-241','FS-253','SADR082 SALIB DIV TO VICT','Victoria_Drive_Weir']
        years = [2060,2070,2080,2090,2100]
        model_version = 304
        excluded_acronyms_csv = ''
        
    if model == 'LISA':   
        model_output_folder = r"J:\SEWER_AREA_MODELS\LISA\04_ANALYSIS_WORK\RAWN_From_Model"
        model_path = r"J:\SEWER_AREA_MODELS\LISA\04_ANALYSIS_WORK\RAWN_From_Model\Lisa_Base.sqlite"
        pop_book = r"J:\SEWER_AREA_MODELS\LISA\04_ANALYSIS_WORK\RAWN_From_Model\LISA_Master_Population.xlsx"
        pop_sheet = 'Export'
        scenario = '2021_Network'
        wwtp_muid = 'WWTP_Outlet'
        line_exclusions = []
        years = [2060,2070]
        model_version = 304
        excluded_acronyms_csv = ''   
        

    #Do not change the lines below
    sewer_area = model 
    global_output_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder 
    
    #Warn user if run_dissolve or run_jpg is False
    if run_dissolve == False or run_jpg == False:
        if MessageBox(None, b'run_dissolve and/or run_jpg is set to False.\n\nThis is only allowed in special cases!\n\nContinue?', b'Warning', 4) == 7:
            MessageBox(None, b"Please set both run_dissolve and run_jpg to True.", b'Info', 0)
            raise ValueError("The user chose to end the execution.")
        else:
            pass
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 3', b'Error', 0)
    raise ValueError("Error")


In [173]:
#Permanent cell 4

#Set up column names

try:
    
    categories = ['res','com','ind','inst','infl','infi']

    mpf_col_dict = {}

    area_col_dict = {}
    area_col_dict['res'] = 'Area_Res'
    area_col_dict['com'] = 'Area_Com'
    area_col_dict['ind'] = 'Area_Ind'
    area_col_dict['inst'] = 'Area_Inst'
    area_col_dict['ini'] = 'Area_Total'

    per_unit_dict = {}
    per_unit_dict['res'] = 320
    per_unit_dict['com'] = 33700 
    per_unit_dict['ind'] = 56200
    per_unit_dict['inst'] = 33700
    per_unit_dict['infl'] = 5600
    per_unit_dict['infi'] = 5600

    unit_dict = {}
    unit_dict['res'] = 'L/c/d'
    unit_dict['com'] = 'L/ha/d'
    unit_dict['ind'] = 'L/ha/d'
    unit_dict['inst'] = 'L/ha/d'
    unit_dict['infl'] = 'L/ha/d'


    header_dict = {}
    # header_dict['gen'] = ['GENERAL INFO',['TYPE','MODELID','CATCHMENT','ID','YEAR','LOCATION']]
    header_dict['gen'] = ['GENERAL INFO',['TYPE','CATCHMENT','YEAR','LOCATION']]
    header_dict['res'] = ['RESIDENTIAL',['AREA (Ha)','POPULATION','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['com'] = ['COMMERCIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['ind'] = ['INDUSTRIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['inst'] = ['INSTITUTIONAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['ini'] = ['INFLOW / INFILTRATION',['AREA (Ha)','INFLOW (L/s)','INFILTRATION (L/s)']]
    header_dict['flow'] = ['FLOWS',['AVG. SAN. FLOW (L/s)','ADWF (L/s)','PWWF (L/s)']]

    avg_calc_dict = {}
    #Items: [Keyword (upper Excel header),Type ('lower Excel header'),Average(lower Excel header),Unit flow cell address, quantifyer column]
    avg_calc_dict['res'] = ['RESIDENTIAL','POPULATION','AVG. FLOW (L/s)','$D$3','H']
    avg_calc_dict['com'] = ['COMMERCIAL','AREA (Ha)','AVG. FLOW (L/s)','$D$4','K']
    avg_calc_dict['ind'] = ['INDUSTRIAL','AREA (Ha)','AVG. FLOW (L/s)','$D$5','N']
    avg_calc_dict['inst'] = ['INSTITUTIONAL','AREA (Ha)','AVG. FLOW (L/s)','$D$6','Q']
    avg_calc_dict['infl'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFLOW (L/s)','$D$7','T']
    avg_calc_dict['infi'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFILTRATION (L/s)','$D$7','T']

    header_tuples = []
    for header in header_dict:
        for sub_header in (header_dict[header][1]):
            header_tuples.append((header_dict[header][0],sub_header))
    header_tuples

    # columns_multiindex = pd.MultiIndex.from_tuples(header_tuples,names=['Category', 'Subcategory'])
    columns_multiindex = pd.MultiIndex.from_tuples(header_tuples)
    df_template = pd.DataFrame(columns=columns_multiindex)

    info_list = []
    for item in unit_dict:
        info_list.append([avg_calc_dict[item][0],per_unit_dict[item],unit_dict[item]])
    info_df = pd.DataFrame(info_list,columns=['DESCRIPTION','AVG. FLOW','UNITS'])
    info_df.set_index('DESCRIPTION',inplace=True)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 4', b'Error', 0)
    raise ValueError("Error")

In [174]:
#Permanent cell 5

#Import model data

try:
    node_types = {}
    node_types[1] = 'Manhole'
    node_types[2] = 'Basin'
    node_types[3] = 'Outlet'
    node_types[4] = 'Junction'
    node_types[5] = 'Soakaway'
    node_types[6] = 'River Junction'

    sql = "SELECT max(AltID) FROM msm_Loadpoint WHERE Active = 1 AND enabled = 1"
    altid = sql_to_df(sql,model_path).iloc[0,0]

    sql = "SELECT max(AltID) FROM msm_Link WHERE Active = 1 AND enabled = 1"
    altid = sql_to_df(sql,model_path).iloc[0,0]

    if altid == 0:
        active_scenario = 'Base'
    else:
        sql = "SELECT MUID FROM m_ScenarioManagementAlternative WHERE GroupID = 'CS_Network' AND AltID = " + str(altid)
        active_scenario = sql_to_df(sql,model_path).iloc[0,0]

    if scenario != active_scenario:
        raise ValueError(f'Scenario {scenario} was requested in the user input but scenario {active_scenario} is active in the model')
    
    sql = "SELECT msm_Catchcon.catchid AS Catchment, msm_Catchcon.nodeid AS Connected_Node FROM msm_Catchcon INNER JOIN msm_catchment "
    sql += "ON msm_Catchcon.catchid = msm_catchment.muid WHERE msm_Catchment.nettypeno <> 2 AND msm_Catchcon.Active = 1 AND msm_catchment.Active = 1 "
    sql += "AND msm_catchment.enabled = 1"
    catchments = sql_to_df(sql,model_path)

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], uplevel AS Outlet_Level FROM msm_Link WHERE Active = 1 AND enabled = 1"
    lines = sql_to_df(sql,model_path)

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Orifice WHERE Active = 1 AND enabled = 1"
    orifices = sql_to_df(sql,model_path)
    lines = pd.concat([lines,orifices])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Valve WHERE Active = 1 AND enabled = 1"
    valves = sql_to_df(sql,model_path)
    lines = pd.concat([lines,valves])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], crestlevel AS Outlet_Level FROM msm_Weir WHERE Active = 1 AND enabled = 1"
    weirs = sql_to_df(sql,model_path)
    lines = pd.concat([lines,weirs])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], startlevel AS Outlet_Level FROM msm_Pump WHERE Active = 1 AND enabled = 1"
    pumps = sql_to_df(sql,model_path)
    lines = pd.concat([lines,pumps])

    lines['Outlet_Level'].fillna(-9999, inplace=True)
    
    lines = lines[~lines['MUID'].isin(line_exclusions)]
    
    excluded_acronyms = []
    if excluded_acronyms_csv != '':
        excluded_acronyms = pd.read_csv(excluded_acronyms_csv)
        excluded_acronyms = list(excluded_acronyms.iloc[:, 0])

    sql = "SELECT muid, UPPER(acronym) AS acronym, UPPER(owner) AS owner, UPPER(assetname) AS assetname, to_outfall FROM msm_Node WHERE active = 1 AND enabled = 1"
    node_id_df = sql_to_df(sql,model_path)
    node_id_df = node_id_df[(((node_id_df.assetname.str[:2]=='MH') & (node_id_df.acronym.notna()) & (node_id_df.acronym!='') & 
        (node_id_df.to_outfall!=1) & (node_id_df.owner=='GV') & (~node_id_df.acronym.isin(excluded_acronyms))) | (node_id_df.muid==wwtp_muid))]
    
    node_id_df.rename(columns={'muid':'Node'},inplace=True)
    node_id_df['ID'] = node_id_df.acronym + '_' + node_id_df.assetname
    node_id_df.loc[node_id_df['Node'] == wwtp_muid, 'ID'] = 'WWTP'
    node_id_df = node_id_df[['Node','ID']]
    
    duplicate_ids = node_id_df[node_id_df.duplicated('ID', keep=False)]
    node_id_df.loc[duplicate_ids.index, 'ID'] = node_id_df['ID'] + '_(' + node_id_df['Node'] + ')'
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 5', b'Error', 0)
    raise ValueError("Error")

In [175]:
#Permanent cell 6

#Import population

try:
    def warning_message(message):
        if MessageBox(None, message.encode('utf-8'), b'Warning', 4) == 7:
            MessageBox(None, b"Please report the issue to the Master Population File admin.", b'Info', 0)
            raise ValueError("The user chose to end the execution.")
        else:
            pass
            
    pop_df = pd.read_excel(pop_book,sheet_name=pop_sheet,dtype={'Catchment': str})#[['Catchment','Year','Pop_Total']]
    pop_df.rename(columns={"Pop_Total": "Population"},inplace=True)
    pop_df = pop_df[['Catchment','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Population','Area_ResLD','Area_ResHD','Area_Mixed','Area_Com','Area_Ind','Area_Inst']]
    pop_df.fillna(0, inplace=True) #Fill NA with 0 or the sum of all will be NA
    
    if transfer_mixed_area_to_commercial:
        pop_df.Area_Com = pop_df.Area_Com + pop_df.Area_Mixed
        pop_df.Area_Mixed = 0
    
    for induced_value in induced_values:
        catchment = induced_value[0]
        column = induced_value[1] 
        value = induced_value[2]
        pop_df.loc[pop_df.Catchment == catchment,column] += value
        
        
    pop_df['Area_Res'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed
    pop_df['Area_Total'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed + pop_df.Area_Com + pop_df.Area_Ind + pop_df.Area_Inst
    pop_df['Population_Sum_Check'] = pop_df.Pop_ResLD + pop_df.Pop_ResHD + pop_df.Pop_Mixed
        
    pop_sum_total_col = int(pop_df.Population.sum())
    pop_sum_sub_cols = int(pop_df.Pop_ResLD.sum() + pop_df.Pop_ResHD.sum() + pop_df.Pop_Mixed.sum())
    pop_df['Key'] = sewer_area + '@' + pop_df.Catchment + '@' + pop_df['Year'].astype(str)
    pop_df.set_index('Key',inplace=True)

    #Check if Pop_Total = 'Pop_ResLD' + 'Pop_ResHD' + 'Pop_Mixed' (sum of all rows)
    if pop_sum_total_col != pop_sum_sub_cols:
        message = f"Warning. The sum of 'Population' in 'Pop_Total' ({pop_sum_total_col:,}) is different than the sum of 'Pop_ResLD' + 'Pop_ResHD' + 'Pop_Mixed' ({pop_sum_sub_cols:,})."
        message += "\n\n'Pop_Total' will be used.\n\nContinue?"
        warning_message(message)
     
    #Check if some rows have population but 0 area
    res_area_check_df = pop_df[(pop_df.Population > 0) & (pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed == 0)]
    if len(res_area_check_df) > 0:
        warning_message(f"{len(res_area_check_df):,} out of {len(pop_df):,} rows have population but 0 residential area.\n\nPrint dataframe 'res_area_check_df' to see all.\n\nContinue?")
    
    #Check is some catchment/year combinations are not found.
    catchment_years = []
    for muid in list(catchments.Catchment.unique()):
        for year in years:
            catchment_years.append([muid,year])
    catchment_year_df = pd.DataFrame(catchment_years,columns=(['Catchment','Year']))
    merged = catchment_year_df.merge(pop_df[['Catchment', 'Year']], on=['Catchment', 'Year'], how='left', indicator=True)
    not_founds = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
    if len(not_founds) > 0:
        message = "The following catchment/year combinations are not found:\n\n"
        for index, row in not_founds[:20].iterrows():
            message += row[0] + ', ' + str(row[1]) + '.\n'
        if len(not_founds) > 20:
             message += "and more, print variable 'not_founds' to see the rest."                                                 
        message += '\n\nContinue?'
        warning_message(message)
                                                                      
        
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 6', b'Error', 0)
    raise ValueError("Error")

In [176]:
#Permanent cell 7
#Trace the model

try:
    accumulated_catchment_set = set()
    accumulated_node_set = set()

    for index1, row1 in catchments.iterrows():
        catchment = row1['Catchment']
        nodes = [row1['Connected_Node']]
        start_node = row1['Connected_Node']
        steps = 0
        

        accumulated_catchment_set.add((start_node,catchment))

        while steps <= max_steps:
            steps += 1
            downstream_df = lines[lines['From'].isin(nodes)]  

            if len(downstream_df) > 0:
                nodes = list(downstream_df.To.unique())

                nodes = [node for node in nodes if len(node)>0]
                for node in nodes:
                    accumulated_catchment_set.add((node,catchment))       
            else:
                break
            if steps == max_steps:
                
#                 catchment_mus.append(catchment)
#                 print('skipped for ' + catchment)
#                 break
                
                
                raise ValueError("Maximum steps were reached, indicating a loop. Last node traced is '" + node + "'")
                
                

            accumulated_catchment_set.add((node,catchment))

    accumulation_df = pd.DataFrame(accumulated_catchment_set,columns=['Node','Catchment'])
    accumulation_df = pd.merge(accumulation_df,node_id_df,how='inner',on=['Node'])
    data = {
        ('GENERAL INFO', 'CATCHMENT'): accumulation_df.Catchment,
        ('GENERAL INFO', 'NODE'): accumulation_df.Node,
        ('GENERAL INFO', 'ID'): accumulation_df.ID,
    }

    # Create a DataFrame with MultiIndex columns
    accumulation_df = pd.DataFrame(data)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 7', b'Error', 0)
    raise ValueError("Error")



In [177]:
#Permanent cell 8
#Calculate RAWN

try:
    catchments = list(pop_df.Catchment.unique())

    catchment_df = df_template.copy()
    for catchment in catchments:
        for year in years:
            key = model + '@' + catchment + '@' + str(year)
            catchment_df.loc[key,('GENERAL INFO','TYPE')] = 'Manhole'
            catchment_df.loc[key,('GENERAL INFO','CATCHMENT')] = catchment
            catchment_df.loc[key,('GENERAL INFO','YEAR')] = year
            catchment_df.loc[key,('GENERAL INFO','LOCATION')] = model
            for area_col_dict_key in area_col_dict:
                catchment_df.loc[key,(header_dict[area_col_dict_key][0],'AREA (Ha)')] = pop_df.loc[key,area_col_dict[area_col_dict_key]]
            catchment_df.loc[key,('RESIDENTIAL','POPULATION')] = pop_df.loc[key,'Population']
            san_flow = 0
            adwf = 0
            for avg_calc_dict_key in avg_calc_dict:
                input1 = catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][1])]
                input2 = per_unit_dict[avg_calc_dict_key]
                avg_flow = input1 * input2 / 86400
                if avg_calc_dict_key not in ['infl','infi']:
                    san_flow += avg_flow
                if avg_calc_dict_key not in ['infl']:
                    adwf += avg_flow    
                catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][2])] = avg_flow
            catchment_df.loc[key,('FLOWS','AVG. SAN. FLOW (L/s)')] = san_flow
            catchment_df.loc[key,('FLOWS','ADWF (L/s)')] = adwf


    catchment_node_df = accumulation_df.merge(catchment_df,on=[('GENERAL INFO','CATCHMENT')],how='inner')
    node_df = catchment_node_df.copy()
    node_df.drop(columns=[('GENERAL INFO','CATCHMENT')],inplace=True)
    node_df = node_df.groupby([('GENERAL INFO','NODE'),('GENERAL INFO','TYPE'),('GENERAL INFO','YEAR'),('GENERAL INFO','LOCATION'),('GENERAL INFO','ID')]).sum()
    node_df.reset_index(inplace=True)
    node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] = np.maximum((1 + 14 / (4 + (node_df[('RESIDENTIAL','POPULATION')] / 1000) ** 0.5)),1.5) * node_df[('RESIDENTIAL','AVG. FLOW (L/s)')]
    node_df[('COMMERCIAL','PEAK FLOW (L/s)')] = np.maximum((1 + 14 / (4 + (per_unit_dict['com'] * node_df[('COMMERCIAL','AREA (Ha)')]/(per_unit_dict['res'] * 1000)) ** 0.5))*0.8,1.5)*node_df[('COMMERCIAL','AVG. FLOW (L/s)')]
    node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] = np.maximum((1 + 14 / (4 + (per_unit_dict['inst'] * node_df[('INSTITUTIONAL','AREA (Ha)')] / (per_unit_dict['res'] * 1000)) ** 0.5)),1.5) * node_df[('INSTITUTIONAL','AVG. FLOW (L/s)')]

    
    mask = node_df[('INDUSTRIAL', 'AREA (Ha)')] != 0 #Avoid error from log(0)
    node_df.loc[mask, ('INDUSTRIAL', 'PEAK FLOW (L/s)')] = np.maximum(
        0.8 * (
            1 + 14 / (
                4 + (node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] * per_unit_dict['ind'] / (per_unit_dict['res'] * 1000)) ** 0.5
            )
        ) * np.where(
            node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] < 121,
            1.7,
            2.505 - 0.1673 * np.log(node_df[('INDUSTRIAL', 'AREA (Ha)')][mask])
        ), 
        1.5
    ) * node_df[('INDUSTRIAL', 'AVG. FLOW (L/s)')][mask]
    
    node_df[('FLOWS','PWWF (L/s)')] = node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] + node_df[('COMMERCIAL','PEAK FLOW (L/s)')] \
        + node_df[('INDUSTRIAL','PEAK FLOW (L/s)')] + node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] \
        + node_df[('INFLOW / INFILTRATION','INFLOW (L/s)')] + node_df[('INFLOW / INFILTRATION','INFILTRATION (L/s)')]
    
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 8', b'Error', 0)
    raise ValueError("Error")



In [178]:
#Permanent cell 9

#Import GIS from the model
#Note that if the RAWN database does not already exist with the layers inside, then it will be created 
#but in this casethe map symbology will be reset and Map Series reset as well, creating extra work.
#An extra word of caution: The first import of e.g. msm_Catchment sets the maximum extend, meaning if you import NSSA in a fresh database,
#then later import FSA, then many catchments will not be imported due to being outside the extend of the old import.
#In the current database, an initial model with added elements outside the extend of all MV models was used to prime the layer. 
#If the database is not maintained, this will need to be done again.
#Efforts to programmatically increase the layer extend were unsuccessful but may later get resolved.


try:

    out_path = global_output_folder + '\\' + gdb_name

    if not os.path.isdir(out_path):
        arcpy.management.CreateFileGDB(global_output_folder, gdb_name)

    arcpy.env.workspace = out_path
    sr = arcpy.SpatialReference(26910)

    layers = ['msm_CatchCon','msm_Catchment','msm_Link','msm_Node','msm_Pump','msm_Weir','msm_Orifice','msm_Valve']

    for layer in layers:

        arcpy.management.MakeFeatureLayer(model_path + '\\' + layer, "temp_layer", "Active = 1")

        if arcpy.Exists(out_path + '\\' + layer):
            print('Appending ' + layer)
            arcpy.management.DeleteFeatures(layer)
            arcpy.management.Append("temp_layer", layer, "NO_TEST")
        else:  
            print('Creating ' + layer)

            arcpy.conversion.FeatureClassToFeatureClass("temp_layer", out_path, layer + '_Test')
            if layer == 'msm_Catchment':
                arcpy.management.AddField('msm_catchment', "Drains_To", "TEXT")
            arcpy.DefineProjection_management(layer, sr)

        arcpy.management.Delete("temp_layer")

        arcpy.env.addOutputsToMap = False
        arcpy.Project_management('msm_Node', 'msm_Node_Google',arcpy.SpatialReference(4326))
        centroids = arcpy.da.FeatureClassToNumPyArray('msm_Node_Google', ("MUID","SHAPE@X","SHAPE@Y"))
        centroids = centroids.tolist()
        centroids_df = pd.DataFrame(centroids, columns =['MUID','X','Y'])
        centroids_df.set_index('MUID',inplace=True)

            
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 9', b'Error', 0)
    raise ValueError("Error")


Appending msm_CatchCon
Appending msm_Catchment
Appending msm_Link
Appending msm_Node
Appending msm_Pump
Appending msm_Weir
Appending msm_Orifice
Appending msm_Valve


In [179]:
#Permanent cell 10

#Create merge_df (realizing the terms 'merge' and 'dissolve' are used here but they mean the same), 
#minimizing the number of computation heavy dissolves that need to be done.
#It prevents the duplicate effort of merging the same catchments together multiple times,
#by moving downstream and reusing upstream merges for downstream merges.

try:
    merge_set = set()

    rank_df = accumulation_df[[('GENERAL INFO','NODE'),('GENERAL INFO','CATCHMENT')]].groupby([('GENERAL INFO','NODE')]).count()

    rank_df.columns = ['Catchment_Count']
    max_catchments = max(rank_df.Catchment_Count)
    rank_df.sort_values(by=['Catchment_Count'],inplace=True)

    catchment_list = []
    merge_set = set()
    for index, row in rank_df.iterrows():

        catchments = list(accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==index][('GENERAL INFO','CATCHMENT')].unique())
        catchments = tuple(sorted(catchments))
        catchment_list.append(catchments)
        merge_set.add(catchments)


    rank_df['Catchments'] = catchment_list
    rank_df['Node'] = rank_df.index


    merge_list = []
    for i, catchments in enumerate(merge_set):
        merge_id = 'Merge_ID_' + str(i)
        merge_list.append([merge_id,catchments])

    merge_df = pd.DataFrame(merge_list,columns=['Merge_ID','Catchments'])
    merge_df['Catchment_Count'] = merge_df['Catchments'].apply(len)
    merge_df.sort_values(by=['Catchment_Count'],ascending=False,inplace=True)
    merge_df.reset_index(inplace=True,drop=True)

    simpler_merge = []
    for index1, row1 in merge_df.iterrows():
        catchments1 = list(row1['Catchments'])
        for index2, row2 in merge_df[index1+1:].iterrows():
            catchments2 = row2['Catchments']

            if len(catchments1) >= len(catchments2):
                if all(item in catchments1 for item in catchments2):
                    catchments1 = [catchment for catchment in catchments1 if catchment not in catchments2]
                    catchments1.append(row2['Merge_ID'])
        simpler_merge.append(catchments1)

    merge_df['To_Dissolve'] = simpler_merge
    merge_df.sort_values(by=['Catchment_Count'],inplace=True)
    merge_df.reset_index(inplace=True,drop=True)

    rank_df = pd.merge(rank_df,merge_df[['Merge_ID','Catchments']], on=['Catchments'],how='inner')
    rank_df.set_index('Node',inplace=True)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 10', b'Error', 0)
    raise ValueError("Error")



In [180]:
#Permanent cell 11

#Run dissolve (also referred to as merge).

try:
    if run_dissolve:
        out_path = global_output_folder + '\\' + gdb_name
        arcpy.env.workspace = out_path  
        arcpy.env.addOutputsToMap = False
        if run_dissolve:
            if arcpy.Exists(gdb_name_dissolve):
                arcpy.management.Delete(gdb_name_dissolve)
            arcpy.management.CreateFileGDB(global_output_folder, gdb_name_dissolve)
            dissolve_path = global_output_folder + '\\' + gdb_name_dissolve
            arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment', dissolve_path, 'Node_Catchment')
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Drains_To", "TEXT")
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID", "TEXT")
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "TEXT")
            arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID", "!muid!", "PYTHON3")
            for index, row in merge_df.iterrows():
                arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "''", "PYTHON3")
                nodes = list(rank_df[rank_df.Merge_ID==row["Merge_ID"]].index)
                print(f'Dissolving for {row["Merge_ID"]}, {index} of {max(merge_df.index)} at time {datetime.datetime.now()}')
                if row['Catchment_Count'] == 1:
                    arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
                    where_clause = f"muid = '{row['To_Dissolve'][0]}'"
                    arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
                    arcpy.management.CalculateField("temp_layer", "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")
                    arcpy.conversion.FeatureClassToFeatureClass('temp_layer', dissolve_path, 'Dissolve_Temp')
                    arcpy.management.Delete("temp_layer")
                    
                else:
                    arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
                    catchments = row['To_Dissolve']
                    catchments_sql = ', '.join([f"'{muid}'" for muid in catchments])
                    where_clause = f"Merge_ID in ({catchments_sql})"
                    arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
                    arcpy.management.CalculateField("temp_layer", "Merge_ID_Temp", f"'{row['Merge_ID']}'", "PYTHON3")
                    arcpy.management.Dissolve("temp_layer",dissolve_path + '\\Dissolve_Temp', "Merge_ID_Temp", "", "MULTI_PART")
                    arcpy.management.Delete("temp_layer")
                    arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")

                for node in nodes:
                    arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Drains_To", f"'{node}'", "PYTHON3")
                    arcpy.management.Append(dissolve_path + '\\Dissolve_Temp', dissolve_path + '\\Node_Catchment', "NO_TEST")

                arcpy.management.Delete(dissolve_path + '\\Dissolve_Temp')



        #Delete the features without a Drains_To
        arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
        where_clause = f"Drains_To IS NULL"
        arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
        arcpy.management.DeleteFeatures("temp_layer")  
        arcpy.management.Delete("temp_layer")

        #Append the features into the official Node_Catchment layer
        arcpy.management.MakeFeatureLayer("Node_Catchment", "Temp_Layer")
        arcpy.management.DeleteFeatures("Temp_Layer")
        arcpy.management.Append(dissolve_path + '\\Node_Catchment', "Temp_Layer", "NO_TEST")
        arcpy.management.Delete("temp_layer")
        
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 11', b'Error', 0)
    raise ValueError("Error")




Dissolving for Merge_ID_244, 0 of 268 at time 2024-08-08 15:51:31.011506
Dissolving for Merge_ID_182, 1 of 268 at time 2024-08-08 15:52:00.174187
Dissolving for Merge_ID_257, 2 of 268 at time 2024-08-08 15:52:26.070394
Dissolving for Merge_ID_33, 3 of 268 at time 2024-08-08 15:52:50.983187
Dissolving for Merge_ID_215, 4 of 268 at time 2024-08-08 15:53:52.655612
Dissolving for Merge_ID_224, 5 of 268 at time 2024-08-08 15:54:28.839561
Dissolving for Merge_ID_46, 6 of 268 at time 2024-08-08 15:54:47.744858
Dissolving for Merge_ID_227, 7 of 268 at time 2024-08-08 15:55:07.026499
Dissolving for Merge_ID_211, 8 of 268 at time 2024-08-08 15:56:30.035444
Dissolving for Merge_ID_229, 9 of 268 at time 2024-08-08 15:56:47.423353
Dissolving for Merge_ID_74, 10 of 268 at time 2024-08-08 15:57:17.631991
Dissolving for Merge_ID_259, 11 of 268 at time 2024-08-08 15:57:34.778195
Dissolving for Merge_ID_101, 12 of 268 at time 2024-08-08 15:57:50.606677
Dissolving for Merge_ID_232, 13 of 268 at time 2024

Dissolving for Merge_ID_230, 111 of 268 at time 2024-08-08 16:42:00.831415
Dissolving for Merge_ID_136, 112 of 268 at time 2024-08-08 16:42:14.284896
Dissolving for Merge_ID_66, 113 of 268 at time 2024-08-08 16:42:28.471012
Dissolving for Merge_ID_121, 114 of 268 at time 2024-08-08 16:43:06.736021
Dissolving for Merge_ID_220, 115 of 268 at time 2024-08-08 16:43:20.505619
Dissolving for Merge_ID_180, 116 of 268 at time 2024-08-08 16:43:34.707612
Dissolving for Merge_ID_22, 117 of 268 at time 2024-08-08 16:43:48.753463
Dissolving for Merge_ID_204, 118 of 268 at time 2024-08-08 16:44:07.041194
Dissolving for Merge_ID_245, 119 of 268 at time 2024-08-08 16:44:20.652648
Dissolving for Merge_ID_67, 120 of 268 at time 2024-08-08 16:44:38.656119
Dissolving for Merge_ID_3, 121 of 268 at time 2024-08-08 16:45:12.619192
Dissolving for Merge_ID_55, 122 of 268 at time 2024-08-08 16:45:30.413472
Dissolving for Merge_ID_235, 123 of 268 at time 2024-08-08 16:45:52.614784
Dissolving for Merge_ID_42, 124

Dissolving for Merge_ID_233, 220 of 268 at time 2024-08-08 17:22:07.133111
Dissolving for Merge_ID_134, 221 of 268 at time 2024-08-08 17:22:22.301989
Dissolving for Merge_ID_195, 222 of 268 at time 2024-08-08 17:22:42.011021
Dissolving for Merge_ID_43, 223 of 268 at time 2024-08-08 17:23:01.592937
Dissolving for Merge_ID_165, 224 of 268 at time 2024-08-08 17:23:25.375696
Dissolving for Merge_ID_185, 225 of 268 at time 2024-08-08 17:24:19.750443
Dissolving for Merge_ID_1, 226 of 268 at time 2024-08-08 17:24:35.039432
Dissolving for Merge_ID_247, 227 of 268 at time 2024-08-08 17:24:50.142249
Dissolving for Merge_ID_10, 228 of 268 at time 2024-08-08 17:25:10.737092
Dissolving for Merge_ID_15, 229 of 268 at time 2024-08-08 17:25:26.180221
Dissolving for Merge_ID_250, 230 of 268 at time 2024-08-08 17:25:41.695416
Dissolving for Merge_ID_115, 231 of 268 at time 2024-08-08 17:26:01.277331
Dissolving for Merge_ID_164, 232 of 268 at time 2024-08-08 17:26:21.023397
Dissolving for Merge_ID_140, 2

In [181]:
#Permanent cell 12

#Export jpgs. 
#This process was too heavy to run inside this notebook, causing failed exports and freeze of the program.
#It was therefore moved to an external script, called from here by writing a batch file and executing it.

try:
    
    if run_jpg:
       
        aprx = arcpy.mp.ArcGISProject("CURRENT")
        project_path = aprx.filePath
        project_folder = os.path.dirname(project_path)

        jpg_folder = model_output_folder + r'\jpg'
        if not os.path.isdir(jpg_folder): os.makedirs(jpg_folder) 

        jpg_script = project_folder + '\\JPG_Subprocess.py'
        bat_file_path = project_folder + '\\Execute_JPG.bat'
        bat_file = open(bat_file_path, "w")
        python_installation = sys.executable
        python_installation = os.path.dirname(sys.executable) + r'\Python\envs\arcgispro-py3\python.exe'

        bat_file_text = '@echo off\n'
        bat_file_text += 'set PYTHON_PATH="' + python_installation + '"\n'
        bat_file_text += 'set SCRIPT_PATH="JPG_Subprocess.py"\n'
        bat_file_text += 'set ARG1="' + project_path + '"\n'
        bat_file_text += 'set ARG2="' + jpg_folder + '"\n'
        bat_file_text += '%PYTHON_PATH% %SCRIPT_PATH% %ARG1% %ARG2%\n'

        bat_file.write(bat_file_text)
        bat_file.close()
        result = subprocess.call([bat_file_path]) 

        if result == 1: #Error
            raise ValueError("The sub process threw an error. Please Locate the bat file: " + bat_file_path + ", open it in notepad, \
            then add a new line and type in letters only: Pause. Double click the bat file to run it and it will show the error.")

        print("Export complete.")

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 12', b'Error', 0)
    raise ValueError("Error")



Export complete.


In [182]:
#Permanent cell 13

#Create spreadsheets

try:
    hex_blue = "ADD8E6"
    hex_yellow = "FFFACD"
    border_style = Side(style='thin', color='000000')
    border = Border(top=border_style, bottom=border_style, left=border_style, right=border_style)
    border_style_none = Side(style=None, color='000000')
    border_none = Border(top=border_style_none, bottom=border_style_none, left=border_style_none, right=border_style_none)
    
    username = os.getlogin()
    date_created = str(datetime.datetime.today()).split(' ')[0]
    
    excel_folder = model_output_folder + '\\Excel'
    img_folder = model_output_folder + '\\jpg'
    backup_folder = f'{excel_folder}\\Backup_{date_created}_{model}_V{model_version}_{pop_sheet}'
    
    if not os.path.isdir(excel_folder): os.makedirs(excel_folder) 
    if not os.path.isdir(img_folder): os.makedirs(img_folder) 
    if not os.path.isdir(backup_folder): os.makedirs(backup_folder) 
    for id in node_df[('GENERAL INFO','ID')].unique():    
        node_single_df = node_df[node_df[('GENERAL INFO','ID')]==id].copy()
        node_single_df.reset_index(drop=True,inplace=True)
       
        if use_formula: #Replace spreadsheet values with formulas
            value_startrow = 17
            for i in node_single_df.index:
                currentrow = i + value_startrow -1
                for avg_calc_dict_key in avg_calc_dict:
#                     currentrow = i + value_startrow
                    inputs = avg_calc_dict[avg_calc_dict_key]
                    header = inputs[0]
                    subheader = inputs[2]
                    unitflow_ref = inputs[3]
                    col_ref = inputs[4]
                    formula = f'{r"="}{unitflow_ref}*{col_ref}{currentrow}/86400'
                    node_single_df.loc[i,(header,subheader)] = formula                
                
                node_single_df.loc[i,('RESIDENTIAL','PEAK FLOW (L/s)')] = f'{r"="}MAX(1.5,(1+(14/(4+((H{currentrow}/1000)^0.5)))))*I{currentrow}'
                node_single_df.loc[i,('COMMERCIAL','PEAK FLOW (L/s)')] = \
                    f'{r"="}MAX(1.5,(1+(14/(4+((({avg_calc_dict["com"][3]}*K{currentrow})/({avg_calc_dict["res"][3]}*1000))^0.5))))*0.8)*L{currentrow}'
                
                ind_formula = f'{r"="}MAX(1.5,0.8*(1+((14)/(4+(((N{currentrow}*{avg_calc_dict["ind"][3]}/{avg_calc_dict["res"][3]}'
                ind_formula += f')/1000)^0.5))))*IF(N{currentrow}<121,1.7,(2.505-0.1673*LN(N{currentrow}))))*O{currentrow}'
                node_single_df.loc[i,('INDUSTRIAL','PEAK FLOW (L/s)')] = ind_formula
                
                node_single_df.loc[i,('INSTITUTIONAL','PEAK FLOW (L/s)')] = \
                    f'{r"="}MAX(1.5,(1+(14/(4+((({avg_calc_dict["inst"][3]}*Q{currentrow})/({avg_calc_dict["res"][3]}*1000))^0.5)))))*R{currentrow}'

                node_single_df.loc[i,('INFLOW / INFILTRATION','AREA (Ha)')] = f'{r"="}G{currentrow}+K{currentrow}+N{currentrow}+Q{currentrow}'
                node_single_df.loc[i,('INFLOW / INFILTRATION','INFLOW (L/s)')] = f'{r"="}T{currentrow}*{avg_calc_dict["infl"][3]}/86400'
                node_single_df.loc[i,('INFLOW / INFILTRATION','INFILTRATION (L/s)')] = f'{r"="}T{currentrow}*{avg_calc_dict["infi"][3]}/86400'
                
                node_single_df.loc[i,('FLOWS','AVG. SAN. FLOW (L/s)')] = f'{r"="}I{currentrow}+L{currentrow}+O{currentrow}+R{currentrow}'
                node_single_df.loc[i,('FLOWS','ADWF (L/s)')] = f'{r"="}W{currentrow}+V{currentrow}'
                node_single_df.loc[i,('FLOWS','PWWF (L/s)')] = \
                    f'{r"="}J{currentrow}+M{currentrow}+P{currentrow}+S{currentrow}+U{currentrow}+V{currentrow}'
                
        id = id if not '/' in id else id.replace('/','_')
        id = id.replace('\\','_')
        id = id.upper()
        muid = node_single_df.iloc[0,0]

        sheetpath = excel_folder + "\\" + id + ".xlsx"
        startrow = 13
        with pd.ExcelWriter(sheetpath) as writer:
            node_single_df.to_excel(writer, sheet_name=id,startrow=startrow)
            info_df.to_excel(writer, sheet_name=id,startrow=1,startcol=2)

            workbook = writer.book
            workbook.create_sheet("Map")

        workbook = load_workbook(sheetpath)    
        sheet1 = workbook[id]

        #Format infobox
        merged_range = sheet1.merged_cells
        for col in sheet1.iter_cols(min_col=3,max_col=5):
            for cell in col[1:2]:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
            for cell in col[2:7]:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = PatternFill(start_color=hex_yellow, end_color=hex_yellow, fill_type="solid")
                cell.border = border
        sheet1.column_dimensions['C'].width = 22 
        sheet1.column_dimensions['D'].width = 11 

        #Remove index
        for row in sheet1.iter_rows():
            for cell in row[:1]:
                cell.value = ''
                cell.border = border_none
        #Format main table header rows
        merged_range = sheet1.merged_cells
        for col in sheet1.iter_cols(min_col=2):
            for cell in col[startrow+1:startrow+2]:
                cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)
                cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
            for cell in col[startrow:startrow+1]:
                if cell.coordinate in sheet1.merged_cells:
                    cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")        

        sheet1.column_dimensions['C'].width = 23 
        sheet1.column_dimensions['D'].width = 11   
        sheet1.column_dimensions['F'].width = 13
        sheet1.column_dimensions['H'].width = 13  
        sheet1.column_dimensions['V'].width = 13
        
        #Delete empty row between header and data
        sheet1.delete_rows(16)
        
        #Find the minimum factor in the sheet. Array formulas do not work with openpyxl, do divisions one by one instead
        cols = [['I','J'],['L','M'],['O','P'],['R','S']]
        rows = list(range(16,currentrow + 1))      
        formula = r'=ROUND(MIN('
        for col in cols:
            for row in rows:
                formula += f'IFERROR({col[1]}{row}/{col[0]}{row},999),'
        formula = formula[:-1] + '),2)'        
        sheet1['J9'] = formula
        sheet1['H9'] = 'Lowest peaking factor:'
        sheet1['H10'] = 'Lowest peaking factors at lower limit 1.5 highlighted in yellow.'
                      
        #Color code cells where peaking facor is at minimum.
        format_formula = 'J16/I16<1.501'
        format_range = ''
        for col in cols:
            format_range += f'{col[1]}16:{col[1]}{currentrow} '
        format_range = format_range[:-1] #Remove last space.               
        fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
        rule = FormulaRule(formula=[format_formula], fill=fill)
        sheet1.conditional_formatting.add(format_range, rule)
        
        decimals = [['#,##0','H'],['#,##0.0','IJLMOPRSUVWXY'],['#,##0.00','GKNQT']]
        for decimal in decimals:
            for col in decimal[1]:
                for row in range(16,currentrow+1):
                    sheet1[f'{col}{row}'].number_format = decimal[0]
        
        display_name = "Open in Google Maps"
        google_map_string = 'https://maps.google.com/?q='
        google_map_string += str(centroids_df.loc[muid,'Y']) + ', '
        google_map_string += str(centroids_df.loc[muid,'X']) 

        # Adding a hyperlink with a display name
        cell = sheet1.cell(row=10, column=3, value=display_name)
        cell.hyperlink = google_map_string
        cell.font = Font(color="0000FF", underline="single") 
                      
        sheet = workbook["Map"]
        
        #Add source info
        source_infos = []
        source_infos.append(['Created by:',username])
        source_infos.append(['Date:',date_created])
        source_infos.append(['Model area:',model])
        source_infos.append(['Model version:',model_version])
        source_infos.append(['Population file:',os.path.basename(pop_book)])
        source_infos.append(['Population sheet:',pop_sheet ])
        
        for i, source_info in enumerate(source_infos):
            sheet1[f'H{i+2}']  = source_info[0]
            sheet1[f'J{i+2}']  = source_info[1]
        
        for row in sheet1['H2:H9']:
            for cell in row:
                cell.font = Font(bold=True)
        

        # Add an image to the sheet
        img_path = img_folder + '\\' + muid + '.jpg'  # Replace with the path to your image
        img = Image(img_path)

        sheet.add_image(img, 'B3')
        
        cell = sheet.cell(row=1, column=2, value=display_name)
        cell.hyperlink = google_map_string
        cell.font = Font(color="0000FF", underline="single") 

        workbook.save(sheetpath)
        shutil.copy(sheetpath,backup_folder + "\\" + id + ".xlsx")


except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 13', b'Error', 0)
    raise ValueError("Error")


In [183]:
message = 'All cells ran successfully.\n\nPlease check the following folder (not its backup subfolders) for obsolete sheets (from previous rounds not overwritten) and delete them. Sort by date to see which ones.\n\n' + model_output_folder + '\\Excel'
MessageBox(None,message.encode('utf-8'), b'Done', 0)

1